In [61]:
import torch as t
from torch import einsum
from einops import rearrange, reduce, repeat
import bert_tests
import math

In [67]:
def raw_attention_scores(
    token_activations: t.Tensor, #[batch_size, seq_length, hidden_size], 
    num_heads: int, 
    project_query, #: function(Tensor[..., 768] -> Tensor[..., num_heads*head_size]), 
    project_key, #: function((Tensor[..., 768]) -> Tensor[..., num_heads*head_size]) -> 
    #Tensor[batch_size, head_num, key_token: seq_length, query_token: seq_length]
):
    #key * query
    batch_s, s_l, hidden_s = token_activations.size()
    query = project_query(token_activations)
    query = rearrange(query, 'b s (n h) -> b n s h', n=num_heads)
    
    key = project_key(token_activations)
    # print(key.size())
    key = rearrange(key, 'b s (n h) -> b n s h', n=num_heads)
    _,_,_,head_s = key.size()

    raw_scores = einsum('bnsh,bnzh->bnzs', query, key)
    print(raw_scores.size())
    print(num_heads)
    return raw_scores / math.sqrt(head_s)
#     t.nn.Softmax(
    
#     break
    
    # return attention_score

#ok, batch size 2, seq_length 3 and hidden size 768, 12 heads
bert_tests.test_attention_pattern_fn(raw_attention_scores)

torch.Size([2, 12, 3, 3])
12
attention pattern raw MATCH!!!!!!!!
 SHAPE (2, 12, 3, 3) MEAN: -0.01231 STD: 0.1219 VALS [-0.1191 0.1944 0.007139 0.03108 -0.1979 -0.3329 0.06184 0.1247 0.126 -0.1378...]


In [115]:
def bert_attention(
token_activations: t.Tensor, #[batch_size, seq_length, hidden_size (768)], 
num_heads: int, 
attention_pattern: t.Tensor, #[batch_size,num_heads, seq_length, seq_length], 
project_value,#func( (Tensor[..., hidden_size (768)]) -> Tensor[..., 12*64] ), 
project_output #func( (Tensor[..., 12*64]) -> Tensor[..., 768] )
):
#-> Tensor[batch_size, seq_length, hidden_size]
    #attention_score: t[batch_size,num_heads, seq_length, seq_length], 
    #Naive attempt
    values = project_value(token_activations)
    values = rearrange(values, \
                       'b sql (num_h vec_length) -> b num_h sql vec_length',\
                      vec_length=64)
    
    soft_max = t.nn.Softmax(dim=-2)
    #These are the scores me think
    scores = soft_max(attention_pattern)
    
    
    
    print(scores.size())
    print(values.size())
    
    #now multiply score into value
    score_times_value = einsum('bnsz,bnhs -> bs', scores, values)
    print(score_times_value.size())
    outputs = project_output(score_times_value)
    return outputs
    
bert_tests.test_attention_fn(bert_attention)

torch.Size([2, 12, 3, 3])
torch.Size([2, 12, 3, 64])


RuntimeError: einsum(): operands do not broadcast with remapped shapes [original->remapped]: [2, 12, 3, 3]->[2, 3, 1, 12, 3] [2, 12, 3, 64]->[2, 64, 3, 12, 1]